# **ALPHA VOLATILITY GENERATION - SYSTEMATIC TRADING STRATEGIES PROJECT**

# **Data Preprocessing**

In [1]:
import yfinance as yf

In [2]:
import numpy as np
from scipy.stats import norm

In [3]:
import pandas as pd

# Charger le fichier CSV
df1 = pd.read_csv("aapl_2016_2020.csv")
df2 = pd.read_csv("aapl_2021_2023.csv")


C:\Users\Jason\AppData\Local\Temp\ipykernel_5572\876459571.py:4: DtypeWarning: Columns (8,9,10,11,12,15,17,18,20,21,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("aapl_2016_2020.csv")
C:\Users\Jason\AppData\Local\Temp\ipykernel_5572\876459571.py:5: DtypeWarning: Columns (15,17,18,20,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("aapl_2021_2023.csv")


In [4]:
df_price = pd.read_csv("Apple_Stock_Price_History.csv")

In [5]:
df_price.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %'], dtype='object')

In [6]:
for col in df_price.columns:
    print(col, type(df_price[col].iloc[0]))

Date <class 'str'>
Price <class 'numpy.float64'>
Open <class 'numpy.float64'>
High <class 'numpy.float64'>
Low <class 'numpy.float64'>
Vol. <class 'str'>
Change % <class 'str'>


In [7]:
print(df_price.iloc[0])

Date        08/01/2025
Price           202.38
Open            210.87
High            213.57
Low             201.51
Vol.            90.81M
Change %        -2.50%
Name: 0, dtype: object


In [8]:
# Convert the df_price['Vol.'] in float
df_price['Vol.'] = df_price['Vol.'].replace(
    {'M': 'e6', 'B': 'e9', 'K': 'e3'}, regex=True
).astype(float)

In [9]:
# Print the different date we have
unique_dates = df_price['Date'].unique()

unique_dates_sorted = sorted(unique_dates)

for date in unique_dates_sorted[:10]:
    print(date)

01/02/2018
01/02/2019
01/02/2020
01/02/2024
01/02/2025
01/03/2017
01/03/2018
01/03/2019
01/03/2020
01/03/2022


In [10]:
# Convert in format aaaa-mm-dd
df_price['Date'] = pd.to_datetime(df_price['Date'], format='%m/%d/%Y')
df_price['Date'] = df_price['Date'].dt.strftime(' %Y-%m-%d')

In [11]:
# Print the different date we have
unique_dates = df_price['Date'].unique()

unique_dates_sorted = sorted(unique_dates)

for date in unique_dates_sorted[:10]:
    print(date)

 2016-01-04
 2016-01-05
 2016-01-06
 2016-01-07
 2016-01-08
 2016-01-11
 2016-01-12
 2016-01-13
 2016-01-14
 2016-01-15


The data includes all option prices between April 28, 2016 and August 1, 2025. 

This database will actually be used to complete the missing stock prices in the database that mainly lists option prices, especially from March 2023 onwards. We will use this database to add prices between April 3, 2023 and December 20, 2024, which correspond to the longest maturity dates.

In [12]:
# We sort in chronological order by the dates
df_price = df_price.sort_values(by='Date').reset_index(drop=True)

In [13]:
# Let's check missing value
mask_na_vol = df_price['Vol.'].isna()

for i in df_price[mask_na_vol].index:
    print(f"Date : {df_price.loc[i, 'Date']}")

Date :  2016-02-27


After checking the official stock price data, it appears that February 27, 2016 is not a trading day. So we will remove that row

In [14]:
df_price = df_price[df_price['Date'] != " 2016-02-27"].reset_index(drop=True)
# We check that the row was successfully removed.”
print(df_price[df_price['Date'] == " 2016-02-27"])

Empty DataFrame
Columns: [Date, Price, Open, High, Low, Vol., Change %]
Index: []


To make it easier to work with dates, we will assign each date a unique integer, essentially creating a one-to-one mapping between dates and integers. This integer code will be stored in the variable j_quote.

In [15]:
date_to_int = {date: i for i, date in enumerate(sorted(df_price['Date'].unique()))}
df_price['j_quote'] = df_price['Date'].map(date_to_int)

In [16]:
for i in range(10):
    print(f"date : {df_price['Date'][i]}, jour : {df_price['j_quote'][i]}")

date :  2016-01-04, jour : 0
date :  2016-01-05, jour : 1
date :  2016-01-06, jour : 2
date :  2016-01-07, jour : 3
date :  2016-01-08, jour : 4
date :  2016-01-11, jour : 5
date :  2016-01-12, jour : 6
date :  2016-01-13, jour : 7
date :  2016-01-14, jour : 8
date :  2016-01-15, jour : 9


The following dataset **df1** and **df2** are those with the options

In [17]:
df1.shape

(1015352, 33)

We have 1015352 options (call and put) 
There are 33 variables in df data set

In [18]:
df2.shape

(548163, 33)

We have 548163 options (call and put)
There are 33 variables in df2 data set

In [19]:
variable = df1.columns.tolist()
print("there are ", len(variable), "variables")

there are  33 variables


In [20]:
variable2 = df2.columns.tolist()
print("there are ", len(variable2), "variables")

there are  33 variables


In [21]:
variable == variable2

True

There are the same variables label for df and df2. So we can concatenate them

In [22]:
df1.columns

Index(['[QUOTE_UNIXTIME]', ' [QUOTE_READTIME]', ' [QUOTE_DATE]',
       ' [QUOTE_TIME_HOURS]', ' [UNDERLYING_LAST]', ' [EXPIRE_DATE]',
       ' [EXPIRE_UNIX]', ' [DTE]', ' [C_DELTA]', ' [C_GAMMA]', ' [C_VEGA]',
       ' [C_THETA]', ' [C_RHO]', ' [C_IV]', ' [C_VOLUME]', ' [C_LAST]',
       ' [C_SIZE]', ' [C_BID]', ' [C_ASK]', ' [STRIKE]', ' [P_BID]',
       ' [P_ASK]', ' [P_SIZE]', ' [P_LAST]', ' [P_DELTA]', ' [P_GAMMA]',
       ' [P_VEGA]', ' [P_THETA]', ' [P_RHO]', ' [P_IV]', ' [P_VOLUME]',
       ' [STRIKE_DISTANCE]', ' [STRIKE_DISTANCE_PCT]'],
      dtype='object')

In [ ]:
# TYPE OF THE VARIABLE 
for col in df1.columns:
    print(col, type(df1[col].iloc[0]))

In [25]:
# Print the different date we have
unique_dates = df1[' [QUOTE_DATE]'].unique()

unique_dates_sorted = sorted(unique_dates)

print(unique_dates_sorted[0])
print(unique_dates_sorted[-1])

 2016-01-04
 2020-12-31


Options from 4 january 2016 to 31 december 2020

In [26]:
# Print the different date we have
unique_dates = df2[' [QUOTE_DATE]'].unique()

unique_dates_sorted = sorted(unique_dates)

print(unique_dates_sorted[0])
print(unique_dates_sorted[-1])

 2021-01-04
 2023-03-31


The datasets df and df2 contain only the options observed between January 4, 2016, and March 31, 2023.

Let's concatenate them : 

In [27]:
df = pd.concat([df1, df2], ignore_index=True)
df.shape

(1563515, 33)

In [28]:
# Print the different date we have
unique_dates = df[' [QUOTE_DATE]'].unique()

unique_dates_sorted = sorted(unique_dates)

print(unique_dates_sorted[0])
print(unique_dates_sorted[-1])

 2016-01-04
 2023-03-31


In [29]:
# We sort in chronological order by the dates
df_sorted = df.sort_values(by=' [QUOTE_DATE]').reset_index(drop=True)

Let’s look at the dates in df_price (the stock price data) and in df_sorted (the option data). We want to find the dates that are present in one dataset but not in the other, and vice versa.

In [30]:
dates_price = set(df_price['Date'].unique())
dates_quote = set(df_sorted[' [QUOTE_DATE]'].unique())

# On trouve les dates présentes dans df_price mais pas dans df_sorted
dates_missing = dates_price - dates_quote
dates_missing = {d for d in dates_missing if d <= ' 2023-03-31'}

dates_missing_inverse = dates_quote - dates_price

In [31]:
# The dates that are exclusive to df_price, meaning they appear in df_price 
# but not in df_sorted.
dates_missing

{' 2016-04-18',
 ' 2017-09-27',
 ' 2019-03-01',
 ' 2020-06-18',
 ' 2020-09-29',
 ' 2020-11-27',
 ' 2020-12-24',
 ' 2022-05-11',
 ' 2022-06-17',
 ' 2022-06-24',
 ' 2022-12-16'}

In [33]:
# The dates that are exclusive to df_sorted, don't appear in df_price.
dates_missing_inverse

{' 2018-12-05',
 ' 2022-01-17',
 ' 2022-02-21',
 ' 2022-04-15',
 ' 2022-05-30',
 ' 2022-06-20',
 ' 2022-07-04',
 ' 2022-09-05',
 ' 2022-11-24',
 ' 2022-12-26'}

After a manual check, it appears that the data in df_sorted is incomplete; some dates are missing for the period from January 4, 2016 to March 31, 2023. The dates exclusive to df_price do indeed exist.

We will manually check the consistency of the dates and prices that are exclusive to df_sorted

In [34]:
df_filtered = df_sorted[df_sorted[' [QUOTE_DATE]'].isin(dates_missing_inverse)]
df_unique = df_filtered[[' [QUOTE_DATE]', ' [UNDERLYING_LAST]']].drop_duplicates()

for _, row in df_unique.iterrows():
    print(f"QUOTE_DATE: {row[' [QUOTE_DATE]']}, UNDERLYING_LAST: {row[' [UNDERLYING_LAST]']}")


QUOTE_DATE:  2018-12-05, UNDERLYING_LAST: 176.69
QUOTE_DATE:  2022-01-17, UNDERLYING_LAST: 173.07
QUOTE_DATE:  2022-02-21, UNDERLYING_LAST: 167.3
QUOTE_DATE:  2022-04-15, UNDERLYING_LAST: 165.29
QUOTE_DATE:  2022-05-30, UNDERLYING_LAST: 149.64
QUOTE_DATE:  2022-06-20, UNDERLYING_LAST: 131.56
QUOTE_DATE:  2022-07-04, UNDERLYING_LAST: 138.93
QUOTE_DATE:  2022-09-05, UNDERLYING_LAST: 155.81
QUOTE_DATE:  2022-11-24, UNDERLYING_LAST: 151.07
QUOTE_DATE:  2022-12-26, UNDERLYING_LAST: 131.86


First, we will merge df_price and df_sorted using an inner join on the ‘Date’ and ‘ [QUOTE_DATE]’ columns, and then display the prices for all matching dates.

In [35]:
df_merged = pd.merge(
    df_price,
    df_sorted,
    left_on='Date',
    right_on=' [QUOTE_DATE]',
    how='inner'  # ou 'left' si tu veux garder toutes les lignes de df_price
)
 
#print(df_merged.head())

In [36]:
'''
seen = set()

# Parcourir chaque ligne du DataFrame
for _, row in df_merged.iterrows():
    key = (row['Date'], row['Price'], row[' [UNDERLYING_LAST]'])
    if key not in seen:
        seen.add(key)
        print(f"Date: {key[0]}, Price: {key[1]}, UNDERLYING_LAST: {key[2]}")
'''

'\nseen = set()\n\n# Parcourir chaque ligne du DataFrame\nfor _, row in df_merged.iterrows():\n    key = (row[\'Date\'], row[\'Price\'], row[\' [UNDERLYING_LAST]\'])\n    if key not in seen:\n        seen.add(key)\n        print(f"Date: {key[0]}, Price: {key[1]}, UNDERLYING_LAST: {key[2]}")\n'

We will remove from df_sorted all the dates that are in dates_missing_inverse.

In [37]:
print(len(df_sorted))
print(len(dates_missing_inverse ))

1563515
10


In [38]:
df_sorted = df_sorted[~df_sorted[' [QUOTE_DATE]'].isin(dates_missing_inverse)]
# Verification
df_verif = df_sorted[df_sorted[' [QUOTE_DATE]'].isin(dates_missing_inverse)]
print(df_verif)



Empty DataFrame
Columns: [[QUOTE_UNIXTIME],  [QUOTE_READTIME],  [QUOTE_DATE],  [QUOTE_TIME_HOURS],  [UNDERLYING_LAST],  [EXPIRE_DATE],  [EXPIRE_UNIX],  [DTE],  [C_DELTA],  [C_GAMMA],  [C_VEGA],  [C_THETA],  [C_RHO],  [C_IV],  [C_VOLUME],  [C_LAST],  [C_SIZE],  [C_BID],  [C_ASK],  [STRIKE],  [P_BID],  [P_ASK],  [P_SIZE],  [P_LAST],  [P_DELTA],  [P_GAMMA],  [P_VEGA],  [P_THETA],  [P_RHO],  [P_IV],  [P_VOLUME],  [STRIKE_DISTANCE],  [STRIKE_DISTANCE_PCT]]
Index: []

[0 rows x 33 columns]


In [39]:
print(len(df_sorted))
print(len(df_merged))

1554718
1554718


In [40]:
nb_dates_uniques = df_sorted[' [QUOTE_DATE]'].nunique()
print(f"Number of unique date in df_sorted : {nb_dates_uniques}")

nb_dates_uniques_merged = df_merged['Date'].nunique()
print(f"Number of unique date in df_merged : {nb_dates_uniques_merged}")


Number of unique date in df_sorted : 1813
Number of unique date in df_merged : 1813


#### **We will continue with df_merged**

We found out that the stock prices were affected by a split ***by 4***, which affects the reliability of the price values in our df_sorted database. We will identify when the split occurred.

In [ ]:
df_unique_quotes = df_merged.drop_duplicates(subset=[' [QUOTE_DATE]', ' [UNDERLYING_LAST]'])

for i in range(len(df_unique_quotes)):
    print(
        f"QUOTE_DATE: {df_unique_quotes[' [QUOTE_DATE]'].iloc[i]}, "
        f"UNDERLYING_LAST: {df_unique_quotes[' [UNDERLYING_LAST]'].iloc[i]}, "
        f"Price: {df_unique_quotes['Price'].iloc[i]},"
    )


Let's identify the date when the apple stock has been splited

In [41]:
mask = np.round(df_merged[' [UNDERLYING_LAST]'] / df_merged['Price']) == 1

df_filtered = df_merged[mask]

if not df_filtered.empty:
    first_date = df_filtered.iloc[0]['Date']
    print(f"Date of the split : {first_date}")
else:
    print("None")

Date of the split :  2020-08-31


Let's add the variable Price_unsplited in df_price 

In [42]:
df_price['Price_unsplited'] = np.where(
    df_price['Date'] < '2020-08-31',
    df_price['Price'] * 4,
    df_price['Price']
)

In [ ]:
for i in range(len(df_price)):
    print(
        f"Date: {df_price['Date'].iloc[i]}, "
        f"Price: {df_price['Price'].iloc[i]}, "
        f"Price_unsplited: {df_price['Price_unsplited'].iloc[i]}"
    )


In [43]:
df_unique_quotes = df_merged.drop_duplicates(subset=[' [QUOTE_DATE]', ' [UNDERLYING_LAST]'])

for i in range(10):
    print(
        f"QUOTE_DATE: {df_unique_quotes[' [QUOTE_DATE]'].iloc[i]}, "
        f"UNDERLYING_LAST: {df_unique_quotes[' [UNDERLYING_LAST]'].iloc[i]}, "
        f"price: {df_unique_quotes['Price'].iloc[i]}, "
        f"jour: {df_unique_quotes['j_quote'].iloc[i]}, "

    )

QUOTE_DATE:  2016-01-04, UNDERLYING_LAST: 105.35, price: 26.34, jour: 0, 
QUOTE_DATE:  2016-01-05, UNDERLYING_LAST: 102.71, price: 25.68, jour: 1, 
QUOTE_DATE:  2016-01-06, UNDERLYING_LAST: 100.7, price: 25.18, jour: 2, 
QUOTE_DATE:  2016-01-07, UNDERLYING_LAST: 96.48, price: 24.11, jour: 3, 
QUOTE_DATE:  2016-01-08, UNDERLYING_LAST: 96.96, price: 24.24, jour: 4, 
QUOTE_DATE:  2016-01-11, UNDERLYING_LAST: 98.53, price: 24.63, jour: 5, 
QUOTE_DATE:  2016-01-12, UNDERLYING_LAST: 99.96, price: 24.99, jour: 6, 
QUOTE_DATE:  2016-01-13, UNDERLYING_LAST: 97.41, price: 24.35, jour: 7, 
QUOTE_DATE:  2016-01-14, UNDERLYING_LAST: 99.51, price: 24.88, jour: 8, 
QUOTE_DATE:  2016-01-15, UNDERLYING_LAST: 97.13, price: 24.28, jour: 9, 


In [44]:
df_merged.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %', 'j_quote',
       '[QUOTE_UNIXTIME]', ' [QUOTE_READTIME]', ' [QUOTE_DATE]',
       ' [QUOTE_TIME_HOURS]', ' [UNDERLYING_LAST]', ' [EXPIRE_DATE]',
       ' [EXPIRE_UNIX]', ' [DTE]', ' [C_DELTA]', ' [C_GAMMA]', ' [C_VEGA]',
       ' [C_THETA]', ' [C_RHO]', ' [C_IV]', ' [C_VOLUME]', ' [C_LAST]',
       ' [C_SIZE]', ' [C_BID]', ' [C_ASK]', ' [STRIKE]', ' [P_BID]',
       ' [P_ASK]', ' [P_SIZE]', ' [P_LAST]', ' [P_DELTA]', ' [P_GAMMA]',
       ' [P_VEGA]', ' [P_THETA]', ' [P_RHO]', ' [P_IV]', ' [P_VOLUME]',
       ' [STRIKE_DISTANCE]', ' [STRIKE_DISTANCE_PCT]'],
      dtype='object')

To handle calendar inconsistencies such as weekends and holidays, we assign each unique trading date a sequential integer index which we has already done before in the df_price. As we want to compute the day to maturity for each option, this ensures that the difference between two date indices reflects the exact number of trading days between them. Since the dataset spans all trading days chronologically.

Let's add the variable day_to_maturity

In [45]:
j_quote_dict = dict(zip(df_price['Date'], df_price['j_quote']))

def compute_day_to_maturity(row):
    expire_j = j_quote_dict.get(row[' [EXPIRE_DATE]'])
    quote_j = j_quote_dict.get(row[' [QUOTE_DATE]'])
    if expire_j is not None and quote_j is not None:
        return expire_j - quote_j
    else:
        return None 

df_merged['day_to_maturity'] = df_merged.apply(compute_day_to_maturity, axis=1)


In [74]:
df_unique_quotes = df_merged.drop_duplicates(subset=[' [QUOTE_DATE]', ' [UNDERLYING_LAST]'])

for i in range(10):
    print(
        f"jour: {df_unique_quotes['Date'].iloc[i]}, "
        f"maturity: {df_unique_quotes[' [EXPIRE_DATE]'].iloc[i]}, "
        f"day_to_mat: {df_unique_quotes['day_to_maturity'].iloc[i]}, "

    )

jour:  2016-01-04, maturity:  2016-01-22, day_to_mat: 13.0, 
jour:  2016-01-05, maturity:  2016-01-29, day_to_mat: 17.0, 
jour:  2016-01-06, maturity:  2016-02-19, day_to_mat: 30.0, 
jour:  2016-01-07, maturity:  2016-02-19, day_to_mat: 29.0, 
jour:  2016-01-08, maturity:  2016-01-15, day_to_mat: 5.0, 
jour:  2016-01-11, maturity:  2016-10-21, day_to_mat: 198.0, 
jour:  2016-01-12, maturity:  2016-01-15, day_to_mat: 3.0, 
jour:  2016-01-13, maturity:  2016-01-15, day_to_mat: 2.0, 
jour:  2016-01-14, maturity:  2016-02-26, day_to_mat: 29.0, 
jour:  2016-01-15, maturity:  2016-01-29, day_to_mat: 9.0, 


We are going to add the moneyness $\frac{K}{S_t}$

In [47]:
df_merged['moneyness'] =  df_merged[' [STRIKE]']/df_merged[' [UNDERLYING_LAST]']

To perform very detailed analyses, we will focus on at-the-money options where the strike price is as close as possible to the spot price $S_t$. We therefore choose a very narrow moneyness window around 1, with $K/S_t​$ ranging from 0.995 to 1.005.

We will keep only options with maturities between 5 and 30 days. For maturities shorter than 5 days, the data is too noisy to build a reasonably robust alpha on volatility. On the other hand, for longer maturities, long-term dependencies become difficult to identify in a reliable way.

In [48]:
mask = (
    (df_merged['moneyness'] >= 0.995) & 
    (df_merged['moneyness'] <= 1.005) & 
    (df_merged['day_to_maturity'] <= 30) & 
    (df_merged['day_to_maturity'] > 5) 
)

for i, row in df_merged[mask][:10].iterrows():
    print(f"Price: {row['Price']}, STRIKE: {row[' [STRIKE]']}, moneyness: {row['moneyness']}, C_IV: {row[' [C_IV]']}, P_IV: {row[' [P_IV]']}")


Price: 26.34, STRIKE: 105.0, moneyness: 0.9966777408637875, C_IV:  0.343480, P_IV:  0.341990
Price: 26.34, STRIKE: 105.0, moneyness: 0.9966777408637875, C_IV:  0.328110, P_IV:  0.327980
Price: 26.34, STRIKE: 105.0, moneyness: 0.9966777408637875, C_IV:  0.267700, P_IV:  0.268210
Price: 26.34, STRIKE: 105.71, moneyness: 1.0034171808258188, C_IV:  0.262920, P_IV:  0.264580
Price: 26.34, STRIKE: 105.0, moneyness: 0.9966777408637875, C_IV:  0.350750, P_IV:  0.350330
Price: 26.34, STRIKE: 105.0, moneyness: 0.9966777408637875, C_IV:  0.280630, P_IV:  0.278480
Price: 25.68, STRIKE: 103.0, moneyness: 1.0028234835945868, C_IV:  0.367580, P_IV:  0.363440
Price: 25.68, STRIKE: 103.0, moneyness: 1.0028234835945868, C_IV:  0.277350, P_IV:  0.272990
Price: 25.68, STRIKE: 103.0, moneyness: 1.0028234835945868, C_IV:  0.357180, P_IV:  0.353200
Price: 25.68, STRIKE: 102.86, moneyness: 1.0014604225489243, C_IV:  0.363160, P_IV:  0.167700


In [49]:
print(mask.sum())

8971


There are 8971 ATM options in our dataset, based on the criteria we previously defined (moneyness within 100% ±0.5%, time to maturity within 5days to 30days).

In [50]:
df_atm = df_merged[mask].copy()

We will aggregate the implied volatility by averaging the implied vol of the call and that of the put.

In [ ]:
for i in df_atm.columns.tolist():
    print(f'{i} : {type(df_atm[i].iloc[0])}')

In [51]:
df_atm[' [C_IV]'] = pd.to_numeric(df_atm[' [C_IV]'].str.replace('%', '').str.strip(), errors='coerce')
df_atm[' [P_IV]'] = pd.to_numeric(df_atm[' [P_IV]'].str.replace('%', '').str.strip(), errors='coerce')

In [52]:
df_atm[' [C_LAST]'] = pd.to_numeric(df_atm[' [C_LAST]'], errors='coerce')
df_atm[' [P_LAST]'] = pd.to_numeric(df_atm[' [P_LAST]'], errors='coerce')

In [53]:
df_atm['IV'] = ((df_atm[' [C_IV]'] + df_atm[' [P_IV]'])/2).round(5)

In [54]:
count_per_date = df_atm.groupby('Date').size()

for date, count in count_per_date[:10].items():
    print(f"Date : {date} : Number of ATM options : {count}")

mean_count = count_per_date.mean()
print(f"\n Average number of ATM options per date : {mean_count:.2f}")

Date :  2016-01-04 : Number of ATM options : 6
Date :  2016-01-05 : Number of ATM options : 6
Date :  2016-01-06 : Number of ATM options : 5
Date :  2016-01-07 : Number of ATM options : 3
Date :  2016-01-08 : Number of ATM options : 3
Date :  2016-01-11 : Number of ATM options : 4
Date :  2016-01-12 : Number of ATM options : 7
Date :  2016-01-13 : Number of ATM options : 9
Date :  2016-01-14 : Number of ATM options : 8
Date :  2016-01-15 : Number of ATM options : 8

 Average number of ATM options per date : 5.67


In [55]:
n_dates = df_atm['Date'].nunique()
print(f"Different date : {n_dates}")

Different date : 1583


We will add a vol_real variable to df_atm, which will represent the realized volatility over the option’s lifetime. As a reminder, realized volatility is simply the annualized standard deviation of the log returns over the given period.

Indeed, we want to get an idea of how many options in our dataset were priced above or below the realized volatility over their respective maturity periods

In [56]:
vol_real_list = []

price_dates = df_price['Date'].tolist()
price_prices = df_price['Price'].tolist()

for i, row in df_atm.iterrows():
    quote_date = row['Date']  # string
    maturity_days = row['day_to_maturity']
    
    if pd.isna(maturity_days):
        vol_real_list.append(np.nan)
        continue
    
    maturity_days = int(maturity_days)
    
    try:
        start_idx = price_dates.index(quote_date)
    except ValueError:
        vol_real_list.append(np.nan)
        continue
    
    if start_idx + maturity_days >= len(price_prices):
        vol_real_list.append(np.nan)
        continue
    
    price_series = price_prices[start_idx : start_idx + maturity_days + 1]
    
    # Vol real -------------------
    log_returns = np.log(np.array(price_series[1:]) / np.array(price_series[:-1]))
    vol_real = np.sqrt(np.mean(log_returns**2) * 252).round(5)
    # ----------------------------
    
    vol_real_list.append(vol_real)

df_atm['vol_real'] = vol_real_list

In [59]:
for i in range(10):
    print(
        f"jour: {df_atm['Date'].iloc[i]}, "
        f"maturity: {df_atm[' [EXPIRE_DATE]'].iloc[i]}, "
        f"day_to_mat: {df_atm['day_to_maturity'].iloc[i]}, "
        f"C_Last: {df_atm[' [C_LAST]'].iloc[i]}, "
        f"P_Last: {df_atm[' [P_LAST]'].iloc[i]}, "
        f"IV: {df_atm['IV'].iloc[i]}, "
        f"Vol real: {df_atm['vol_real'].iloc[i]}, "

    )


jour:  2016-01-04, maturity:  2016-02-05, day_to_mat: 23.0, C_Last: 4.31, P_Last: 4.95, IV: 0.34274, Vol real: 0.41644, 
jour:  2016-01-04, maturity:  2016-02-12, day_to_mat: 28.0, C_Last: 4.5, P_Last: 5.05, IV: 0.32804, Vol real: 0.37998, 
jour:  2016-01-04, maturity:  2016-01-15, day_to_mat: 9.0, C_Last: 2.22, P_Last: 1.88, IV: 0.26796, Vol real: 0.37948, 
jour:  2016-01-04, maturity:  2016-01-15, day_to_mat: 9.0, C_Last: 1.8, P_Last: 2.25, IV: 0.26375, Vol real: 0.37948, 
jour:  2016-01-04, maturity:  2016-01-29, day_to_mat: 18.0, C_Last: 4.05, P_Last: 3.75, IV: 0.35054, Vol real: 0.44552, 
jour:  2016-01-04, maturity:  2016-01-22, day_to_mat: 13.0, C_Last: 2.79, P_Last: 2.5, IV: 0.27956, Vol real: 0.39194, 
jour:  2016-01-05, maturity:  2016-01-29, day_to_mat: 17.0, C_Last: 3.9, P_Last: 4.06, IV: 0.36551, Vol real: 0.4479, 
jour:  2016-01-05, maturity:  2016-01-22, day_to_mat: 12.0, C_Last: 2.37, P_Last: 2.65, IV: 0.27517, Vol real: 0.39102, 
jour:  2016-01-05, maturity:  2016-02-0

Since each row in our DataFrame corresponds to the quoted prices of both ATM call and put options for a given strike and maturity, we can associate each row with a **Straddle**.

In [60]:
count_above = (df_atm['vol_real'] > df_atm['IV']).sum()
prop_above = count_above / len(df_atm)

# Cas où vol_real < IV
count_below = (df_atm['vol_real'] < df_atm['IV']).sum()
prop_below = count_below / len(df_atm)

# Affichage
print(f"Number of STRADDLE > IV : {count_above} ({prop_above:.2%})")
print(f"Number of STRADDLE < IV : {count_below} ({prop_below:.2%})")

Number of STRADDLE > IV : 3345 (37.29%)
Number of STRADDLE < IV : 5626 (62.71%)


Let's add the price of the Straddle defined as the sum of the atm call and the atm put prices

In [61]:
df_atm['Straddle'] = (df_atm[' [C_LAST]'] + df_atm[' [P_LAST]']).round(2)

----------

## **Delta and Pricer**

In [62]:
def black_scholes_call_price(S, K, T, r, sigma):
    if T <= 0:
        return max(S - K, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2)* T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def black_scholes_put_price(S, K, T, r, sigma):
    if T <= 0:
        return max(K - S, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price

def black_scholes_call_delta(S, K, T, r, sigma):
    if T <= 0:
        return 1.0 if S > K else 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return norm.cdf(d1)

def black_scholes_put_delta(S, K, T, r, sigma):
    if T <= 0:
        return -1.0 if S < K else 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return norm.cdf(d1) - 1 

We chose an interest rate of 0.03

In [63]:
# Call delta
df_atm['call_delta'] = df_atm.apply(lambda row: black_scholes_call_delta(
    S=row[' [UNDERLYING_LAST]'],
    K=row[' [STRIKE]'],
    T=row['day_to_maturity'] / 252,
    r=0.03,
    sigma=row[' [C_IV]']
), axis=1)

# Put delta
df_atm['put_delta'] = df_atm.apply(lambda row: black_scholes_put_delta(
    S=row[' [UNDERLYING_LAST]'],
    K=row[' [STRIKE]'],
    T=row['day_to_maturity'] / 252,
    r=0.03,
    sigma=row[' [P_IV]']
), axis=1)

In [64]:
df_atm['call_delta'] = df_atm['call_delta'].round(5)
df_atm['put_delta'] = df_atm['put_delta'].round(5)

In [65]:
for i in range(10):
    print(
        f"jour: {df_atm['Date'].iloc[i]},  "
        f"maturity:{df_atm[' [EXPIRE_DATE]'].iloc[i]},  "
        f"day_to_mat:{df_atm['day_to_maturity'].iloc[i]},  "
        f"price:{df_atm['Price'].iloc[i]},  "
        f"price_unsplitted:{df_atm[' [UNDERLYING_LAST]'].iloc[i]},  "
        f"strike:{df_atm[' [STRIKE]'].iloc[i]},  "
        f"c_iv:{df_atm[' [C_IV]'].iloc[i]},  "
        f"p_iv:{df_atm[' [P_IV]'].iloc[i]},  "
        f"c_delta_df:{df_atm[' [C_DELTA]'].iloc[i]},  "
        f"p_delta_df:{df_atm[' [P_DELTA]'].iloc[i]},  "
        f"c_delta_num:{df_atm['call_delta'].iloc[i]},  "
        f"p_delta_num:{df_atm['put_delta'].iloc[i]},  "
        f"IV:{df_atm['IV'].iloc[i]},  "
        f"Vol real:{df_atm['vol_real'].iloc[i]},  "

    )


jour:  2016-01-04,  maturity: 2016-02-05,  day_to_mat:23.0,  price:26.34,  price_unsplitted:105.35,  strike:105.0,  c_iv:0.34348,  p_iv:0.34199,  c_delta_df:0.52311,  p_delta_df:-0.47609,  c_delta_num:0.54393,  p_delta_num:-0.45606,  IV:0.34274,  Vol real:0.41644,  
jour:  2016-01-04,  maturity: 2016-02-12,  day_to_mat:28.0,  price:26.34,  price_unsplitted:105.35,  strike:105.0,  c_iv:0.32811,  p_iv:0.32798,  c_delta_df:0.52202,  p_delta_df:-0.47715,  c_delta_num:0.54601,  p_delta_num:-0.45399,  IV:0.32804,  Vol real:0.37998,  
jour:  2016-01-04,  maturity: 2016-01-15,  day_to_mat:9.0,  price:26.34,  price_unsplitted:105.35,  strike:105.0,  c_iv:0.2677,  p_iv:0.26821,  c_delta_df:0.53657,  p_delta_df:-0.46323,  c_delta_num:0.54469,  p_delta_num:-0.45536,  IV:0.26796,  Vol real:0.37948,  
jour:  2016-01-04,  maturity: 2016-01-15,  day_to_mat:9.0,  price:26.34,  price_unsplitted:105.35,  strike:105.71,  c_iv:0.26292,  p_iv:0.26458,  c_delta_df:0.48085,  p_delta_df:-0.51881,  c_delta_num:

---

We will split our dataset into three sets: 70% for training, 15% for validation, and 15% for testing.

In [66]:
n = len(df_atm)
n_train = int(n * 0.70)
n_val = int(n * 0.15)

df_train = df_atm.iloc[:n_train]
df_validation = df_atm.iloc[n_train:n_train + n_val]
df_test = df_atm.iloc[n_train + n_val:]

---

## **Gamma Scalping test**

In [67]:
dates_uniques = df_price['Date'].unique().tolist()

In [68]:
df_price.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %', 'j_quote',
       'Price_unsplited'],
      dtype='object')

In [69]:
date_to_price = dict(zip(df_price['Date'], df_price['Price']))
date_to_price_unsplitted = dict(zip(df_price['Date'], df_price['Price_unsplited']))

In [71]:
r = 0.03
dt = 1/252  # 252 business days in a year
PNL_list = []
iv_vs_realvol = []

for _, row in df_train.iterrows():

    # Filter on wanted date to avoid splited stock problem
    if row['Date'] <= " 2020-07-31":
        price_dict = date_to_price_unsplitted
    elif row['Date'] >= " 2020-09-01":
        price_dict = date_to_price
    else:
        continue 

    date_id1 = dates_uniques.index(row['Date'])
    date_id2 = dates_uniques.index(row[' [EXPIRE_DATE]'])
    list_date = dates_uniques[date_id1:date_id2 + 1]

    iv_call = row[' [C_IV]']
    iv_put = row[' [P_IV]']

    hedge_gain = []
    PNL_total = []

    C_0 = row[' [C_LAST]']
    P_0 = row[' [P_LAST]']

    # Hedging part
    for i in range(1, len(list_date)):
        S_tbefore = price_dict[list_date[i-1]]
        S_t = price_dict[list_date[i]]

        time_to_maturity_before = (len(list_date) - (i - 1)) / 252
        time_to_maturity = (len(list_date) - i) / 252

        C_tbefore = black_scholes_call_price(S_tbefore, row[' [STRIKE]'], time_to_maturity_before, r, iv_call)
        C_t = black_scholes_call_price(S_t, row[' [STRIKE]'], time_to_maturity, r, iv_call)
        P_tbefore = black_scholes_put_price(S_tbefore, row[' [STRIKE]'], time_to_maturity_before, r, iv_put)
        P_t = black_scholes_put_price(S_t, row[' [STRIKE]'], time_to_maturity, r, iv_put)

        delta_call_t = black_scholes_call_delta(S_tbefore, row[' [STRIKE]'], time_to_maturity_before, r, iv_call)
        delta_put_t = black_scholes_put_delta(S_tbefore, row[' [STRIKE]'], time_to_maturity_before, r, iv_put)

        hedge_value = (-delta_call_t * (S_t - S_tbefore) +
                       (delta_call_t * S_tbefore - C_tbefore) * (np.exp(r * dt) - 1)
                       - delta_put_t * (S_t - S_tbefore) +
                       (delta_put_t * S_tbefore - P_tbefore) * (np.exp(r * dt) - 1))

        hedge_gain.append(hedge_value)

        pnl_total_t = (C_t + P_t - C_0 - P_0) + np.sum(hedge_gain[:i])
        PNL_total.append(pnl_total_t)  
        
    PNL_list.append(PNL_total[-1])
    iv_vs_realvol.append(row['vol_real'] - row['IV'])

    print(f"date option: {row['Date']},  PNL: {PNL_total[-1]},  IV: {row['IV']},  vol_real: {row['vol_real']}")


date option:  2016-01-04,  PNL: 1.5976891062764684,  IV: 0.34274,  vol_real: 0.41644
date option:  2016-01-04,  PNL: 1.8849534642713501,  IV: 0.32804,  vol_real: 0.37998
date option:  2016-01-04,  PNL: 1.6336920545867168,  IV: 0.26796,  vol_real: 0.37948
date option:  2016-01-04,  PNL: 1.491090677526445,  IV: 0.26375,  vol_real: 0.37948
date option:  2016-01-04,  PNL: 1.9458455541848023,  IV: 0.35054,  vol_real: 0.44552
date option:  2016-01-04,  PNL: 1.447509206128649,  IV: 0.27956,  vol_real: 0.39194
date option:  2016-01-05,  PNL: 2.19929981270737,  IV: 0.36551,  vol_real: 0.4479
date option:  2016-01-05,  PNL: 1.6055756694724983,  IV: 0.27517,  vol_real: 0.39102
date option:  2016-01-05,  PNL: 2.2733248783093374,  IV: 0.35519,  vol_real: 0.41705
date option:  2016-01-05,  PNL: 1.312388094810251,  IV: 0.26543,  vol_real: 0.37646
date option:  2016-01-05,  PNL: 1.184483480550244,  IV: 0.26082,  vol_real: 0.37646
date option:  2016-01-05,  PNL: 2.0379389629784455,  IV: 0.33924,  vol_r

# **CONCLUSION**

**With the following script, we can see that the PnL from the gamma hedging we backtested on the df_train options clearly depends on the difference between the implied volatility used for pricing and the volatility that was actually realized.**

In [73]:
mean_positive = np.sum([pnl for pnl, diff in zip(PNL_list, iv_vs_realvol) if diff > 0])
mean_negative = np.sum([pnl for pnl, diff in zip(PNL_list, iv_vs_realvol) if diff < 0])

print("PNL given that vol_real > iv:", mean_positive)
print("PNL given that vol_real < iv :", mean_negative)

PNL given that vol_real > iv: 9619.300661573769
PNL given that vol_real < iv : -4474.266709231813
